In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


#import spotify keys
from config import client_id
from config import client_secret

# Creating Access Token for Spotify API

In [2]:
#generate access token for spotify api
#from: https://stmorse.github.io/journal/spotify-api.html

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

#In order to access, we send a properly formed GET request to the API server, with our access_token in the header. Let’s save this header info now, in the following very specific format:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# Requesting Input from User

In [3]:
# need to add instructions or code to remove punctuations for search to work
search_track = input("Please name a song: ")

Please name a song:  the chain


In [4]:
search_artist = input("Please name that song's artist: ")

Please name that song's artist:  fleetwood mac


In [5]:
# region_preference = input("Please select a region: ")

# (Keep cell below for testing track search)

In [6]:
#base URL for Spotify API search
search_url = 'https://api.spotify.com/v1/search'

response = requests.get(search_url + '?q=track%3A' + search_track + '%20artist%3A' + search_artist + '&type=track%2Cartist&limit=1', headers=headers).json()

pprint(response)

# track_id = response['tracks']['items'][0]["id"]
# artist_names = response['tracks']['items'][0]["artists"][0]['name']
# track_name = response['tracks']['items'][0]["name"]

{'artists': {'href': 'https://api.spotify.com/v1/search?query=track%3Athe+chain+artist%3Afleetwood+mac&type=artist&offset=0&limit=1',
             'items': [],
             'limit': 1,
             'next': None,
             'offset': 0,
             'previous': None,
             'total': 0},
 'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3Athe+chain+artist%3Afleetwood+mac&type=track&offset=0&limit=1',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/08GQAI4eElDnROBrJRGE0X'},
                                              'href': 'https://api.spotify.com/v1/artists/08GQAI4eElDnROBrJRGE0X',
                                              'id': '08GQAI4eElDnROBrJRGE0X',
                                              'name': 'Fleetwood Mac',
                                              'type': 'artist',
                                              'uri': 's

# Function to Search for Matching Track Using Spotify API, Create DF of Track Features

In [7]:
def searchTrack(track,artist):
    #base URL for Spotify API search
    search_url = 'https://api.spotify.com/v1/search'
    
    # API search for track & artist input by user
    response = requests.get(search_url + '?q=track%3A' + search_track + '%20artist%3A' + search_artist + '&type=track%2Cartist&limit=1', headers=headers).json()
    
    # storing track id, artist, and track name from API search as variables
    track_id = response['tracks']['items'][0]["id"]
    artist_names = response['tracks']['items'][0]["artists"][0]['name']
    track_name = response['tracks']['items'][0]["name"]
    
    #base URL for Spotify API
    base_url = 'https://api.spotify.com/v1/'
    # API call to collect track features
    track_response = requests.get(base_url + 'audio-features/' + track_id, headers=headers).json()
    
    #empty lists to store song data retrieved from API call
    danceability = []
    energy = []
    key = []
    loudness = []
    mode = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    id_num = []
    duration_mins = []
    time_signature = []

    #fill in track info for each audio feature / empty list
    danceability.append(track_response["danceability"])
    energy.append(track_response["energy"])
    key.append(track_response["key"])
    loudness.append(track_response["loudness"])
    mode.append(track_response["mode"])
    speechiness.append(track_response["speechiness"])
    acousticness.append(track_response["acousticness"])
    instrumentalness.append(track_response["instrumentalness"])
    liveness.append(track_response["liveness"])
    valence.append(track_response["valence"])
    tempo.append(track_response["tempo"])
    id_num.append(track_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    duration_mins.append(round((track_response["duration_ms"] / 60000),2))
    time_signature.append(track_response["time_signature"])    

    #create a dictionary to hold data gathered from the api
    track_dict = {
        "track_id": id_num,
        "artist_names": artist_names,
        "track_name": track_name,
        "danceability": danceability,
        "energy": energy,
        "key": key,
        "loudness": loudness,
        "mode": mode,
        "speechiness": speechiness,
        "acousticness": acousticness,
        "instrumentalness": instrumentalness,
        "liveness": liveness,
        "valence": valence,
        "tempo": tempo,
        "duration_mins": duration_mins,
        "time_signature": time_signature,

    }
    #convert dictionary to a dataframe
    input_track_df = pd.DataFrame(track_dict)
    
    return input_track_df

In [11]:
input_track_df = searchTrack(search_track,search_artist)
input_track_df

,track_id,artist_names,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,5e9TFTbltYBg2xThimr0rU,Fleetwood Mac,The Chain - 2004 Remaster,0.545,0.67,9,-8.81,1,0.0496,0.009,0.000822,0.0451,0.481,151.553,4.5,4


# Reading in CSVs of Regional and Global Top 200 Charts

In [12]:
#import CSVs created in "pulling_chart_track_features.ipynb" and save as individual dfs
features_csv_list = {"Global":"global_df.csv", "Vietnam":"vietnam_df.csv","Venezuela":"venezuela_df.csv",
"Uruguay":"uruguay_df.csv","United Kingdom":"united_kingdom_df.csv","Ukraine":"ukraine_df.csv",
"UAE":"uae_df.csv","Turkey":"turkey_df.csv","Thailand":"thailand_df.csv",
"Taiwan":"taiwan_df.csv","Switzerland":"switzerland_df.csv","Sweden":"sweden_df.csv",
"Spain":"spain_df.csv","South Korea":"south_korea_df.csv","South Africa":"south_africa_df.csv",
"Slovakia":"slovakia_df.csv","Singapore":"singapore_df.csv","Saudi Arabia":"saudi_arabia_df.csv",
"Romania":"romania_df.csv","Portugal":"portugal_df.csv","Poland":"poland_df.csv",
"Philippines":"philippines_df.csv","Peru":"peru_df.csv","Paraguay":"paraguay_df.csv",
"Panama":"panama_df.csv","Pakistan":"pakistan_df.csv","Norway":"norway_df.csv",
"Nigeria":"nigeria_df.csv","Nicaragua":"nicaragua_df.csv","New Zealand":"new_zealand_df.csv",
"Netherlands":"netherlands_df.csv","Mexico":"mexico_df.csv",
"Malaysia":"malaysia_df.csv","Luxembourg":"luxembourg_df.csv","Lithuania":"lithuania_df.csv",
"Latvia":"latvia_df.csv","Kazakhstan":"kazakhstan_df.csv","Japan":"japan_df.csv",
"Italy":"italy_df.csv","Israel":"israel_df.csv","Ireland":"ireland_df.csv",
"Indonesia":"indonesia_df.csv",
"Hungary":"hungary_df.csv","Hong Kong":"hong_kong_df.csv","Honduras":"honduras_df.csv",
"Guatemala":"guatemala_df.csv","Greece":"greece_df.csv","Germany":"germany_df.csv",
"France":"france_df.csv","Finland":"finland_df.csv","Estonia":"estonia_df.csv",
"El Salvador":"el_salvador_df.csv","Egypt":"egypt_df.csv","Ecuador":"ecuador_df.csv",
"Dominican Republic":"dominican_republic_df.csv","Denmark":"denmark_df.csv","Czech Republic":"czech_republic_df.csv",
"Cyprus":"cyprus_df.csv","Costa Rica":"costa_rica_df.csv","Colombia":"colombia_df.csv",
"Chile":"chile_df.csv","Canada":"canada_df.csv","Bulgaria":"bulgaria_df.csv",
"Brazil":"brazil_df.csv","Bolivia":"bolivia_df.csv","Belgium":"belgium_df.csv",
"Belarus":"belarus_df.csv","Austria":"austria_df.csv","Australia":"australia_df.csv",
"Argentina":"argentina_df.csv"}
# currently not working w/ api calls: "morocco":"morocco_df.csv","india":"india_df.csv","iceland":"iceland_df.csv"

d = {}

for key in features_csv_list:
    d[key] = pd.read_csv(f'resources/top_chart_features/{features_csv_list[key]}') 


In [13]:
regions_list = d.keys()
regions_list

dict_keys(['Global', 'Vietnam', 'Venezuela', 'Uruguay', 'United Kingdom', 'Ukraine', 'UAE', 'Turkey', 'Thailand', 'Taiwan', 'Switzerland', 'Sweden', 'Spain', 'South Korea', 'South Africa', 'Slovakia', 'Singapore', 'Saudi Arabia', 'Romania', 'Portugal', 'Poland', 'Philippines', 'Peru', 'Paraguay', 'Panama', 'Pakistan', 'Norway', 'Nigeria', 'Nicaragua', 'New Zealand', 'Netherlands', 'Mexico', 'Malaysia', 'Luxembourg', 'Lithuania', 'Latvia', 'Kazakhstan', 'Japan', 'Italy', 'Israel', 'Ireland', 'Indonesia', 'Hungary', 'Hong Kong', 'Honduras', 'Guatemala', 'Greece', 'Germany', 'France', 'Finland', 'Estonia', 'El Salvador', 'Egypt', 'Ecuador', 'Dominican Republic', 'Denmark', 'Czech Republic', 'Cyprus', 'Costa Rica', 'Colombia', 'Chile', 'Canada', 'Bulgaria', 'Brazil', 'Bolivia', 'Belgium', 'Belarus', 'Austria', 'Australia', 'Argentina'])

In [14]:
len(regions_list)

70

# Function to Recommend 5 Tracks from Selected Top 200 Chart

In [15]:
def recommendSongs(user_track_df, selected_chart_df):
    # create combined df of the track features from the input track and all of the tracks from the selected regional chart
    # row 0 will be the input song
    combined_df = pd.concat([user_track_df,selected_chart_df.loc[:]]).reset_index(drop=True)    
    
    # scale data (only features columns), create df of scaled data
    scaler = StandardScaler()
    chart_scaled = scaler.fit_transform(combined_df.iloc[:, 3:16])
    chart_scaled_df = pd.DataFrame(chart_scaled, columns=combined_df.iloc[:, 3:16].columns)

    # create array for input track data
    array1 = chart_scaled_df.iloc[0,:].to_numpy().reshape(1, -1)
    # create array for selected chart
    array2 = chart_scaled_df.iloc[1:,:].to_numpy()

    # run cosine similarity
    cosine_sim = cosine_similarity(array1, array2)
    
    # create list that is ranked by score, descending order
    sim_scores = list(enumerate(cosine_sim[-1,:]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # create empty list to store recommendations
    ranked_tracks = []

    for i in range(0, 5):
        indx = sim_scores[i][0]
        ranked_tracks.append([combined_df['track_id'].iloc[indx], combined_df['artist_names'].iloc[indx], combined_df['track_name'].iloc[indx], np.round(sim_scores[i][1],decimals=2)])
    
    return ranked_tracks

In [16]:
recommendSongs(input_track_df,d['Global'])

[['27ZZdyTSQWI7Cug2d2PkqV', 'Armani White', 'BILLIE EILISH.', 0.79],
 ['2J2Z1SkXYghSajLibnQHOa', 'Sia', 'Unstoppable', 0.69],
 ['4RvWPyQ5RL0ao9LPZeSouE',
  'Tears For Fears',
  'Everybody Wants To Rule The World',
  0.67],
 ['0VjIjW4GlUZAMYd2vXMi3b', 'The Weeknd', 'Blinding Lights', 0.62],
 ['3uUuGVFu1V7jTQL60S1r8z',
  'Lost Frequencies, Calum Scott',
  'Where Are You Now',
  0.62]]

In [17]:
recommendSongs(input_track_df,d['UAE'])

[['4xqrdfXkTW4T0RauPLv3WA', 'Conan Gray', 'Heather', 0.78],
 ['2kx5sQgmjYug0GiDh1a4sU',
  'Cheriimoya, Sierra Kidd',
  'Living Life, In The Night',
  0.78],
 ['2gMXnyrvIjhVBUZwvLZDMP', 'Lewis Capaldi', 'Before You Go', 0.72],
 ['0VO8gYVDSwM1Qdd2GsMoYK',
  'Swedish House Mafia, The Weeknd',
  'Moth To A Flame (with The Weeknd)',
  0.71],
 ['3sNVsP50132BTNlImLx70i', 'Kanye West', 'Bound 2', 0.68]]